In [71]:
#Nathan removed functions related to terminal commands from given library
from part_ncbiblast_library import *
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import os, sys
import re


Getting sequence to BLAST from user, optional email and name of BLAST

In [72]:
#test seq: MIDNAYHIDS (4 hits Uniprot, 0 hits Phytozome)
#test seq 2: MARPFTLSLLSLCLLLSAWSCFGGSSSTNRFNICQLNSLNALKPDHRVETDGGLVETWSSRHPELECAGVTVTRRTLYRNGFQMPSYSPYSQMIMAIQGKGALGLALSGCAETYEEPAKESSSSSQKPSDSHQKIRQFDQGHVMLIPRGVPFWIFNTGDEPLITVTLLDTSSEDNQLDQSPREFYLAGNPDIEHPEAMKEKQQQQAEEEGGNVLSGFGKRFLARALNIDQDTANKLISPDDEMKQIVKLKEGLSVISPKWQGQQEDEDEDDDDEDEDESVSRPSRRPSHGKRVHKEEETVVEPYPHGKHVHKEVEKEVEPLPPRKHVHKEEEKEIEPLPPRRSRHHHDEGEDEGEEEEKPRARRTRGPTPSPKGEGHRGVEEEDESEDTKGHKTRHEKTRHEKSWKEHRPEGEDVEKGEAHEEWETRPSKDKPHGSNGLDETICSSKLQFNIARPKGADFYNPKAGRIKNLNSQSLPALQHFGLSAQYVVLYKNGIYSPHWNMDANSVIYVIRGQGQVRVVNNEGIVMFDDELKKGQLLVVPQNFMVAEEAGDQGFEYVVFKTNDNAVTSYLKETFRAFPAEVLVNIYKLKHSQVHDLKYNGNLGPLVNPENSLDQSS
#(250 hits Uniprot, 100 hits Phytozome)
seq = str(input("Protein AA Sequence to Compare: "))
email = (input("What's your email: ") or 'tafsia.shikdar@beyondmeat.com')
title = str(input('Name this run: ') or 'Protein Search')

Setting params for Uniprot API

In [73]:
#Required for uniprot blast. Only input params are email and sequence, everything else is default
params = {
        "program":"blastp",
        "matrix":"PAM30",
        "alignments":"250",
        "scores":"250",
        "exp":"10",
        "filter":"F",
        "gapalign":"true",
        "compstats":"F",
        "align":"0",
        "stype":"protein",
        "sequence": seq,
        "database":"uniprotkb_refprotswissprot"
    }

Running Uniprot BLAST

In [74]:
#Running Uniprot job after asking for a name (otherwise default)
jobId = serviceRun(email, title, params)

Starting Phytozome webscraping/BLAST simultaneously to save time; setting up remotely controlled browser

In [75]:
#object of Options class
op = webdriver.ChromeOptions()
#browser preferences i.e. saving all downloaded files into certain location to find later
p = {'download.default_directory':'C:\\Users\\Tafsia Shikdar\\Downloads\\phytozome_download'}
#add options to browser i.e. where to save downloaded files, and not opening a new window visibly
op.add_experimental_option('prefs', p)
#op.add_argument('headless')
#webdriver (interface for remotely controlling browsers) location, specifically for Google Chrome
service = ChromeService(executable_path='C:/Users/Tafsia Shikdar/Desktop/protein-uniprot-blast/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=op)


Running Phytozome BLAST (using Selenium)

In [76]:
#load phytozome blast webpage
driver.get("https://phytozome-next.jgi.doe.gov/blast-search")
#HTML code showing location of search text box, and clicking into text box
click = WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div/main/div/div/div/div/div/div[1]/div[2]/div/ol/li/span/label/span')))
driver.execute_script("arguments[0].click();", click)
#input sequence into box
search = WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div/main/div/div/div/div/form/fieldset/textarea')))
driver.execute_script("arguments[0].value= '" + seq + "'", search)
#had some issues with inputting sequence - worked after adding and deleting a space after seq is input
search.send_keys(Keys.SPACE)
search.send_keys(Keys.BACK_SPACE)
#HTML code showing location of enter/go button
go = WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div/main/div/div/div/div/form/fieldset/div/ul/li[1]/span')))
driver.execute_script("arguments[0].click();", go)

Waiting for Uniprot BLAST to complete

In [77]:
#This function checks Uniprot job status every 3 seconds until it is done
clientPoll(jobId)
#Retrieving Uniprot result as a json file and loading it in so it can be parsed as dictionary
file = serviceGetResult(jobId, 'json')
results = json.loads(file)

RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING


KeyboardInterrupt: 

Chains are split into alpha and beta chains according to Ryan Prestangen's algorithm to split chains into alpha and beta (assumed for 11S globulin proteins specifically). Let me know if this needs to be updated - in future we can look at machine learning models to replace this manually designed splitting method.
Algorithm: Will not split proteins smaller than 20 amino acids, or if there are no C (cysteine) residues past the first 5 residues in the chain.
First, tries to find an NG terminus (asparagine immediately followed by a threonine) with a C 7 residues after it. If this is found, chain is split 5 amino acids before the C.
Secondly, if such a spot cannot be found, count the number of cysteines in the SECOND HALF of the peptide chain. IF there are more than 2 cysteines in the second half, chain is split 5 spots before the third to last cysteine in the SECOND HALF of protein. Otherwise, if there are only 1 or 2 cysteines, chain is split 5 spots before the first cysteine in the SECOND HALF of protein.
Finally, if there are no cysteines in the second half of the sequence, split chain five spots before the last cysteine in the FIRST HALF of the protein.

In [ ]:
def splitting_chains(protein_seq):
    '''Splits sequence into two chains.
    Parameters: (str) AA sequence
    Returns: (tuple) ((str) Method of Split, 
                      (int) Index for start of second chain OR (str) Reason for not splitting protein)'''
    #will only do this if chain is longer than 20 AA
    size = len(protein_seq)
    if size < 20:
        return ('No Split', 'Protein Too Small to Split')
    ng_indexes = []
    c_indexes = []
    for i in range(size):
        aa = protein_seq[i]
        #cysteines in the first 5 AAs not counted
        if i > 5 and aa == 'C':
            c_indexes.append(i)
        #First method: trying to find a cysteine 7 spots after an NG. If this is found, chain should be 
        #split 2 spots after the NG index/5 spots before the cysteine
        try:
            if aa == 'N' and protein_seq[i+1] == 'G':
                ng_indexes.append(i)
                if protein_seq[i+7] == 'C':
                    return ('NG Terminus at ' + str(i), i+2)
        except:
            continue
    #Second method: Find all cysteines in second half of sequence, and base split on number of Cs found
    #If no Cs, can't split
    c_count = len(c_indexes)
    if c_count == 0:
        return ('No Split', 'Protein Has No Viable Cysteines')
    half = size/2
    #Only using the c_indexes found in second half of protein sequence
    i = 0
    while i < c_count and c_indexes[i] < half:
        i += 1
    second_half_c = c_indexes[i+1:]
    second_c_count = len(second_half_c)
    if second_c_count != 0:
        #If more than 2 cystines in second half of protein seq, chain should be split 5 spots 
        #before the third to last cysteine
        if second_c_count > 2:
            c_loc = second_half_c[-3]
            return ('Third to Last Cysteine in Second Half at ' + str(c_loc), c_loc - 5)
        #If only 1 or 2 cystines in second half of protein seq, chain should be split 5 spots before the first
        #cysteine.
        else:
            c_loc = second_half_c[0]
            return ('First Cysteine in Second Half at ' + str(c_loc), c_loc - 5)
    #Third Method: Split chain 5 spots before last cysteine in first half of protein seq
    c_loc = c_indexes[:i][-1] 
    return ('Last Cysteine in First Half at ' + str(c_loc), c_loc - 5)


In [ ]:
data = {
    'Website': [],
    'Link':[], 
    'Entry': [], 
    'Entry Name': [], 
    'Protein Name': [], 
    'Organism': [], 
    'Gene Name': [], 
    'Length': [], 
    'Sequence': [], 
    'Percent Identity': [],
    'Split Method': [],
    'Chain 1 Sequence': [],
    'Chain 2 Sequence': []
    }

#Creating function to fill in sequence split columns using given sequence and chain_splitting algorithm
def split_chain_func(chain_sequence):
    split_chain_method, split_chain_result = splitting_chains(chain_sequence)
    data['Split Method'].append(split_chain_method)
    if type(split_chain_result) != int:
        data['Chain 1 Sequence'].append(split_chain_result)
        data['Chain 2 Sequence'].append(split_chain_result)
    else:
        data['Chain 1 Sequence'].append(chain_sequence[:split_chain_result])
        data['Chain 2 Sequence'].append(chain_sequence[split_chain_result:])  


Total Results:250


Searching through Uniprot JSON, pulling the data needed into a dataframe

In [ ]:
#Manually searched through retrieved json file to identify keys of the info we needed - may be an easier/less hard coded way to do this via API queries
columns_to_keys = {
    'Link':'hit_url', 
    'Entry': 'hit_acc', 
    'Entry Name': 'hit_id', 
    'Protein Name': 'hit_uni_de', 
    'Organism': 'hit_uni_os', 
    'Gene Name': 'hit_uni_gn', 
    'Length': 'hit_len',
}

#in case info doesn't exist for entry, default message fills the dataframe
def add_info(column):
    result_key = columns_to_keys[column]
    if result_key in hit:
        data[column].append(hit[result_key])
    else:
        data[column].append('No Data Found')

#while building df, will let user know total number of entries and progress
print('Total Uniprot Results:' + str(len(results['hits'])))
#goes through each hit entry to pull the necessary info using manually matched keys to data desired
for hit in results['hits']:
    print('Uniprot' + str(hit['hit_num']))
    data['Website'].append('Uniprot')
    for c in columns_to_keys:
        add_info(c)
    #adding percent identity, unlike other desired data, this one was nested in the original json file
    try:
        data['Percent Identity'].append(hit['hit_hsps'][0]['hsp_identity'])
    except:
        data['Percent Identity'].append('No Data Found')
    #sequence not in original data file, so querying api for specific entry to get that data
    url = 'https://rest.uniprot.org/uniprotkb/' + hit['hit_acc'] + '.json'
    info = json.loads(restRequest(url))
    full_seq = info['sequence']['value'].upper()
    data['Sequence'].append(full_seq)
    #Splitting chain
    split_chain_func(full_seq) 


Uniprot1
Uniprot2
Uniprot3
Uniprot4
Uniprot5
Uniprot6
Uniprot7
Uniprot8
Uniprot9
Uniprot10
Uniprot11
Uniprot12
Uniprot13
Uniprot14
Uniprot15
Uniprot16
Uniprot17
Uniprot18
Uniprot19
Uniprot20
Uniprot21
Uniprot22
Uniprot23
Uniprot24
Uniprot25
Uniprot26
Uniprot27
Uniprot28
Uniprot29
Uniprot30
Uniprot31
Uniprot32
Uniprot33
Uniprot34
Uniprot35
Uniprot36
Uniprot37
Uniprot38
Uniprot39
Uniprot40
Uniprot41
Uniprot42
Uniprot43
Uniprot44
Uniprot45
Uniprot46
Uniprot47
Uniprot48
Uniprot49
Uniprot50
Uniprot51
Uniprot52
Uniprot53
Uniprot54
Uniprot55
Uniprot56
Uniprot57
Uniprot58
Uniprot59
Uniprot60
Uniprot61
Uniprot62
Uniprot63
Uniprot64
Uniprot65
Uniprot66
Uniprot67
Uniprot68
Uniprot69
Uniprot70
Uniprot71
Uniprot72
Uniprot73
Uniprot74
Uniprot75
Uniprot76
Uniprot77
Uniprot78
Uniprot79
Uniprot80
Uniprot81
Uniprot82
Uniprot83
Uniprot84
Uniprot85
Uniprot86
Uniprot87
Uniprot88
Uniprot89
Uniprot90
Uniprot91
Uniprot92
Uniprot93
Uniprot94
Uniprot95
Uniprot96
Uniprot97
Uniprot98
Uniprot99
Uniprot100
Uniprot1

Moving onto Phytozome Blast...first downloading blast results file

In [ ]:
#If there are hits, there will be an export button on phytozome blast results page, otherwise it will search for the 'no hits found' text
export_button = WebDriverWait(driver, 1000).until(EC.any_of(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/main/div/div[1]/div[1]/ul/li[1]/button')), EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div/div/h2'))))
#If no hits, create empty file, otherwise download results
try:
    no_hits = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/h2')
    empty_data = {
    'Views':[], 
    'Protein': [], 
    'Species': [], 
    'E-Value': [], 
    '% identity': [], 
    'Align len': [], 
    'Strands': [], 
    'Query ID': [], 
    'Query from': [],
    'Query to': [],
    'Target from': [],
    'Target to': [],
    'Bitscore': [],
    '# identical': [],
    'Positives': [],
    'Gaps': [],
    'Query len': [],
    'Target len': []
    }
    phy_results = pd.DataFrame(empty_data)
except:
    #Export as TSV file
    driver.execute_script("arguments[0].click();", export_button)
    path = 'C:\\Users\\Tafsia Shikdar\\Downloads\\phytozome_download\\'
    list = os.listdir(path)
    #find the file name by getting the last downloaded file
    times = [os.path.getctime(path + file) for file in list]
    last_file_time = max(times)
    last_file_index = times.index(last_file_time)
    last_file = list[last_file_index]
    #Load into a pandas dataframe
    phy_results = pd.read_csv(path + last_file, sep = '\t').drop('Unnamed: 0', axis = 1)
    

['blastp_job_616585 (1).tsv', 'blastp_job_616585.tsv', 'blastp_job_633883.tsv', 'blastp_job_633957.tsv', 'blastp_job_635781.tsv', 'blastp_job_639705.tsv', 'blastp_job_639805.tsv', 'blastp_job_639889.tsv', 'blastp_job_639912.tsv', 'blastp_job_639941.tsv', 'blastp_job_640337 (1).tsv', 'blastp_job_640337.tsv', 'blastp_job_641493.tsv', 'blastp_job_641530.tsv', 'blastp_job_644826.tsv', 'd10e545c-0055-448a-b858-3cb6e7d8f71c.tmp', 'export.csv', 'GO_terms_for_proteins.json', 'services.json', 'VuZN016.07G092700.functionalAnnotations.tsv']


,Views,Protein,Species,E-value,% identity,Align len,Strands,Query ID,Query from,Query to,Target from,Target to,Bitscore,# identical,Positives,Gaps,Query len,Target len
0,phytozome-next.jgi.doe.gov/report/protein/Vung...,VuZN016.07G092700.1.p,V.unguiculata_ZN016 v1.2,0.000000e+00,85,620,+/+,seq.submitted 2022-08-22T18:41:21.472Z UTC,1,618,1,611,877.085,527,567,11,618,612
1,phytozome-next.jgi.doe.gov/report/protein/Vung...,VuTZ30.07G092400.1.p,V.unguiculata_TZ30 v1.2,0.000000e+00,85,620,+/+,seq.submitted 2022-08-22T18:41:21.472Z UTC,1,618,1,611,877.085,527,567,11,618,612
2,phytozome-next.jgi.doe.gov/report/protein/Vung...,VuCB5-2.07G096300.1.p,V.unguiculata_CB5-2 v1.1,0.000000e+00,85,620,+/+,seq.submitted 2022-08-22T18:41:21.472Z UTC,1,618,1,611,877.085,527,567,11,618,612
3,phytozome-next.jgi.doe.gov/report/protein/Vung...,VuSanzi.07G093900.1.p,V.unguiculata_Sanzi v1.1,0.000000e+00,85,620,+/+,seq.submitted 2022-08-22T18:41:21.472Z UTC,1,618,1,611,877.085,527,567,11,618,612
4,phytozome-next.jgi.doe.gov/report/protein/Vung...,VuUCR779.07G095400.1.p,V.unguiculata_UCR779 v1.1,0.000000e+00,85,620,+/+,seq.submitted 2022-08-22T18:41:21.472Z UTC,1,618,1,611,877.085,527,567,11,618,612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,phytozome-next.jgi.doe.gov/report/protein/Mgut...,MgNONTOL.G0505.1.p,M.guttatus_NONTOL v3.1,6.440000e-34,36,222,+/+,seq.submitted 2022-08-22T18:41:21.472Z UTC,40,258,73,273,143.280,80,124,24,618,495
193,phytozome-next.jgi.doe.gov/report/protein/Acoe...,Aqcoe5G027700.1.p,A.coerulea v3.1,3.310000e-48,46,184,+/+,seq.submitted 2022-08-22T18:41:21.472Z UTC,430,610,288,471,182.956,84,126,3,618,482
194,phytozome-next.jgi.doe.gov/report/protein/Acoe...,Aqcoe5G027700.1.p,A.coerulea v3.1,9.160000e-36,35,243,+/+,seq.submitted 2022-08-22T18:41:21.472Z UTC,34,258,32,274,148.288,86,130,18,618,482
195,phytozome-next.jgi.doe.gov/report/protein/Pami...,FUN_051248-T1,P.amilis v1.0,3.590000e-48,45,190,+/+,seq.submitted 2022-08-22T18:41:21.472Z UTC,415,601,274,463,182.956,86,127,3,618,487


In [ ]:
#Manually linking exported file columns to desired column names
phy_cols_to_keys = {
    'Link':'Views',  
    'Protein Name': 'Protein', 
    'Organism': 'Species', 
    'Gene Name': 'Protein', 
    'Length': 'Target len',
    'Percent Identity': '% identity',
}

num_phy_results = len(phy_results.index)
print('Total Phytozome Results:' + str(num_phy_results))
#goes through each phytozome hit to get the desired data
for row in range(num_phy_results):
    #progress update
    print('Phytozome' + str(row))
    data['Website'].append('Phytozome')
    data['Entry'].append('No Data Found')
    data['Entry Name'].append('No Data Found')
    for phy_col in phy_cols_to_keys:
        data[phy_col].append(phy_results.loc[row][phy_cols_to_keys[phy_col]])
    #Peptide sequence isn't in downloaded file, so need to find on individual webpages
    #Open individul protein webpage
    driver.get('https://' + phy_results.loc[row]['Views'])
    #Click button to reveal peptide seq
    peptide_seq_reveal = WebDriverWait(driver, 1000).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/main/div/div[4]/div[5]/label')))
    driver.execute_script("arguments[0].click();", peptide_seq_reveal)
    #Seq split into multiple elements on web page, so get all elements and join to make one sequence, without the * at the end
    peptide_seq_els = driver.find_elements(By.CLASS_NAME, 'cds')
    peptide_seq = ''.join([e.text for e in peptide_seq_els])
    final_seq = peptide_seq[:-1]
    #Add seq to dataframe
    data['Sequence'].append(final_seq)
    #Split chain according to algorithm
    split_chain_func(final_seq)
    

Phytozome0
Phytozome1
Phytozome2
Phytozome3
Phytozome4
Phytozome5
Phytozome6
Phytozome7
Phytozome8
Phytozome9
Phytozome10
Phytozome11
Phytozome12
Phytozome13
Phytozome14
Phytozome15
Phytozome16
Phytozome17
Phytozome18
Phytozome19
Phytozome20
Phytozome21
Phytozome22
Phytozome23
Phytozome24
Phytozome25
Phytozome26
Phytozome27
Phytozome28
Phytozome29
Phytozome30
Phytozome31
Phytozome32
Phytozome33
Phytozome34
Phytozome35
Phytozome36
Phytozome37
Phytozome38
Phytozome39
Phytozome40
Phytozome41
Phytozome42
Phytozome43
Phytozome44
Phytozome45
Phytozome46
Phytozome47
Phytozome48
Phytozome49
Phytozome50
Phytozome51
Phytozome52
Phytozome53
Phytozome54
Phytozome55
Phytozome56
Phytozome57
Phytozome58
Phytozome59
Phytozome60
Phytozome61
Phytozome62
Phytozome63
Phytozome64
Phytozome65
Phytozome66
Phytozome67
Phytozome68
Phytozome69
Phytozome70
Phytozome71
Phytozome72
Phytozome73
Phytozome74
Phytozome75
Phytozome76
Phytozome77
Phytozome78
Phytozome79
Phytozome80
Phytozome81
Phytozome82
Phytozome83
Ph

Creating final CSV file of data

In [ ]:
df=pd.DataFrame(data)
#Sort dataframe according to highest percentage identity
df = df.sort_values('Percent Identity', ascending=False)
#Renumbering after resorting
df = df.reset_index(drop=True)
#Name file (if name not given, name by first 5 AAs of seq blasted)
if title != 'Protein Search':
    name = title
else:
    name = params['sequence'][:5]
#Export dataframe to CSV and save
df.to_csv(r'C:\Users\Tafsia Shikdar\Desktop\\' + name + '.csv')